In [1]:
from tensorflow.keras.utils import to_categorical
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
seed = 0
np.random.seed(seed)
import tensorflow as tf

tf.random.set_seed(seed)
import os

#os.environ['PATH'] = os.environ['XILINX_VIVADO'] + '/bin:' + os.environ['PATH']

2024-09-13 16:08:22.951768: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 16:08:22.992778: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-09-13 16:08:22.993427: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 16:08:23.798581: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l1
import pandas as pd
from keras.utils import np_utils

import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from keras.models import Sequential
from keras.layers import Dense


In [3]:
df = pd.read_csv('activities_preprocessed_approx.csv')
X = df.values[:,:-1]
Y = df.values[:,-1]

# Scale the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

enc = OneHotEncoder()
Y = enc.fit_transform(Y[:, np.newaxis]).toarray()

X_train, X_test, Y_train, Y_test = train_test_split(X_scaled, Y, test_size=0.3, random_state=42)

print(X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)

(115943, 18) (49690, 18) (115943, 5) (49690, 5)


In [4]:
# Create the model
model = Sequential()
model.add(Dense(20, input_dim=18, activation='relu'))
model.add(Dense(5, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=5, verbose=1)

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f'Loss: {loss}, Accuracy: {accuracy}')


Epoch 1/10
23189/23189 [==============================] - 24s 1ms/step - loss: 0.2240 - accuracy: 0.9325
Epoch 2/10
23189/23189 [==============================] - 23s 981us/step - loss: 0.1105 - accuracy: 0.9705
Epoch 3/10
23189/23189 [==============================] - 22s 969us/step - loss: 0.0946 - accuracy: 0.9751
Epoch 4/10
23189/23189 [==============================] - 23s 971us/step - loss: 0.0875 - accuracy: 0.9775
Epoch 5/10
23189/23189 [==============================] - 23s 973us/step - loss: 0.0814 - accuracy: 0.9787
Epoch 6/10
23189/23189 [==============================] - 23s 970us/step - loss: 0.0782 - accuracy: 0.9796
Epoch 7/10
23189/23189 [==============================] - 22s 970us/step - loss: 0.0748 - accuracy: 0.9805
Epoch 8/10
23189/23189 [==============================] - 22s 968us/step - loss: 0.0728 - accuracy: 0.9813
Epoch 9/10
23189/23189 [==============================] - 23s 970us/step - loss: 0.0705 - accuracy: 0.9816
Epoch 10/10
1553/1553 [================

In [5]:
import plotting
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
y_keras = model.predict(X_test)
print("Accuracy: {}".format(accuracy_score(np.argmax(Y_test, axis=1), np.argmax(y_keras, axis=1))))

1553/1553 [==============================] - 1s 800us/step
Accuracy: 0.9811229623666734


In [11]:
y_test = enc.inverse_transform(Y_test)
y_test = [i[0] for i in y_test]

In [12]:
# Create the C file
with open('Activities_testdata/test_dataset.c', 'w') as f:
    # Write the data samples array
    f.write("float test_data[%d][%d] = {\n" % (X_test.shape[0], X_test.shape[1]))
    for row in X_test:
        f.write("    {%s},\n" % ", ".join(f"{x:.6f}f" for x in row))
    f.write("};\n\n")

    # Write the labels array
    f.write("int test_labels[%d] = {\n" % len(y_test))
    f.write("    %s\n" % ", ".join(map(str, y_test)))
    f.write("};\n")

print("Test dataset has been saved to 'test_dataset.c'")


Test dataset has been saved to 'test_dataset.c'


In [16]:
# Get the weights and biases
layer1_weights = model.layers[0].get_weights()[0].T
layer1_bias = model.layers[0].get_weights()[1]
layer2_weights = model.layers[1].get_weights()[0].T
layer2_bias = model.layers[1].get_weights()[1]

# Write weights and biases to weights.cpp
with open('Activities_testdata/weights.cpp', 'w') as f:
    f.write('#include <vector>\n\n')
    
    f.write('std::vector<std::vector<float>> layer1_weights = {\n')
    for row in layer1_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer1_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer1_bias))
    f.write('};\n\n')

    f.write('std::vector<std::vector<float>> layer2_weights = {\n')
    for row in layer2_weights:
        f.write('    {' + ', '.join(f'{x:.6f}f' for x in row) + '},\n')
    f.write('};\n\n')

    f.write('std::vector<float> layer2_bias = {')
    f.write(', '.join(f'{x:.6f}f' for x in layer2_bias))
    f.write('};\n')

print("Weights and biases have been saved to 'weights.cpp'")


Weights and biases have been saved to 'weights.cpp'


In [14]:
!g++ -std=c++11 Activities_testdata/test.cpp -o Activities_testdata/test_nn

In [15]:
!./Activities_testdata/test_nn

Accuracy: 65.8201%
